# Setup and Data Updates

### Postgres access

Connecting to database:

In [1]:
import psycopg2
import sqlalchemy as sql

In [2]:
pg_dbname = '...'
pg_username = '...'
pg_endpoint = '...rds.amazonaws.com'
pg_port = '...'
pg_password = '...'

pg_engine = sql.create_engine('postgres://{}:{}@{}:{}/{}'.format(
    pg_username,
    pg_password,
    pg_endpoint,
    pg_port,
    pg_dbname))

### Upload new ahrefs data

Takes a backlinks file and a domains file from ahrefs and adds them to the database, including to the list of pages that need to be scraped.

In [ ]:
from everest import upload

In [ ]:
# backlinks_file = 'Data/backlinks_recent_all.csv'
# domains_file = 'Data/domains_all.csv'

In [ ]:
backlinks_file = 'Data/backlinks_recent_all_new.csv'
domains_file = 'Data/domains_all_new.csv'

In [ ]:
upload.upload_ahrefs_data(backlinks_file, domains_file, pg_engine)

### Scrape

In [ ]:
from everest import scrape

In [ ]:
scrape.scrape(pg_engine)

This scrapes urls in scraped_domains table.

Optional parameters are:

1 - scrape_condition sets which domains to scrape, and should be a SQL WHERE string, e.g. default is "WHERE last_scrape_date IS NULL", which scrapes all domains that haven't been scraped before. See below for table schema.

2 - time_limit parameter sets how long the python http request should wait for a response. Longer wait times mean scraping takes longer, but shorter means slow sites are missed. Default is 2 seconds.

The response of the http request is converted to a base64 string in the 'html' column. The text content of any <main> tags (or <body> if no <main> tags) are stored in 'main', also converted to base64 strings, and are used for text clustering. http links are saved as lists, converted to json then encoded in base64 - 'links' gives all links, 'links_main_domains' extracts just the domain, and 'links_main_domains' includes only the domains that are already in the database, e.g. because they've linked to the site.

### Text clustering

Runs the text clustering algorithm. This finds sites with similar text and clusters them together. It does it separately for each language. Non-English languages are essentially instant, but clustering English sites can take anywhere from 10 minutes to 2 hours depending on computer performance.

In [3]:
from everest import cluster

/Users/jeremyward/anaconda3/envs/everest/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/jeremyward/anaconda3/envs/everest/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
cluster.recluster(pg_engine)

Clustering complete for language code he
Clustering complete for language code da
Clustering complete for language code tl
Clustering complete for language code sk
Clustering complete for language code ca
Clustering complete for language code mr
Clustering complete for language code lt
Clustering complete for language code lv
Clustering complete for language code id
Clustering complete for language code zh-cn
Clustering complete for language code ko
Clustering complete for language code tr
Clustering complete for language code en
Clustering complete for language code sw
Clustering complete for language code ja
Clustering complete for language code es
Clustering complete for language code cs
Clustering complete for language code bn
Clustering complete for language code uk
Clustering complete for language code nl
Clustering complete for language code el
Clustering complete for language code af
Clustering complete for language code mk
Clustering complete for language code ro
Clustering co

### Database Table Info

In [29]:
from everest import utils

Database tables are as follows:

In [49]:
utils.frame_from_pg("""SELECT * FROM information_schema.tables
WHERE table_schema = 'public'
""",pg_engine)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,postgres,public,clusters,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
1,postgres,public,backlinks,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
2,postgres,public,domains,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
3,postgres,public,scraped_domains,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
4,postgres,public,url_main_domain_matches,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN


'backlinks' is just the ahrefs backlinks file with the following additions:

'live' says whether it's live or not (which is TRUE iff 'backlink_status' is NULL)

'link_type' records whether it has an anchor or not [other types redacted].

'last_updated' says when the entry was last updated.

'url_main' extracts the url from the link for the purpose of matching it up with the domains in the domains table

In [25]:
utils.frame_from_pg("""SELECT column_name ,data_type
FROM information_schema.columns
WHERE table_name = 'backlinks'
""",pg_engine)

,column_name,data_type
0,domain_rating,bigint
1,url_rating,bigint
2,referring_domains,double precision
3,referring_page_url,text
4,referring_page_title,text
5,internal_links_count,bigint
6,external_links_count,bigint
7,link_url,text
8,textpre,text
9,link_anchor,text


'domains' is just the ahrefs domains file with the addition of 'last_updated', which says when the entry was last updated.

In [50]:
utils.frame_from_pg("""SELECT column_name ,data_type
FROM information_schema.columns
WHERE table_name = 'domains'
""",pg_engine)

,column_name,data_type
0,referring_domain,text
1,domain_rating,bigint
2,ahrefs_rank,double precision
3,total_backlinks,bigint
4,backlinks_for_referring_domain_percent,double precision
5,dofollow_backlinks,bigint
6,dofollow_backlinks_for_referring_domain_percent,double precision
7,first_seen,timestamp without time zone
8,referring_domains,bigint
9,linked_domains,bigint


'scraped_domains' contains the scraping info (including http request, scraping, and parsing errors) for all domains in the ahrefs set whose links are [redacted].

'lang' gives the language detected from page text by python's langdetect package.

'html', 'main', and links columns are as described in scraping section above, encoded as base64.

'live_backlinks' and 'total_backlinks' give the number of live and total backlinks for that domain in the backlinks table. 

'url_https' and 'url_www' give counts of that domain's links in the backlinks table, which is used to determine which protocols to use when requesting the page during scraping.

In [24]:
utils.frame_from_pg("""SELECT column_name ,data_type
FROM information_schema.columns
WHERE table_name = 'scraped_domains'
""",pg_engine)

,column_name,data_type
0,domain,text
1,first_seen,text
2,html,text
3,lang,text
4,last_scrape_code,text
5,last_scrape_date,text
6,last_scrape_parse_error,text
7,last_scrape_request_error,text
8,last_successful_scrape_date,text
9,links,text


'clusters' gives the results of the clustering algorithm - a list of domains, their detected language, which cluster they're in (en04 means cluster 04 in the English language clustering. -1 means no cluster, which applies to most sites since they aren't similar to any other). 

'mean_distance' is the textual distance between sites in that cluster - smaller numbers mean the sites are more similar.

In [27]:
utils.frame_from_pg("""SELECT column_name ,data_type
FROM information_schema.columns
WHERE table_name = 'clusters'
""",pg_engine)

,column_name,data_type
0,domain,text
1,lang,text
2,cluster,text
3,mean_distance,text
